### Working with `selenium`: continued

In [ ]:
%pip install selenium
%pip install webdriver_manager
%pip install pandas
%pip install bs4

Today we will continue to work on the task uploading the addresses of all precinct election commissions of the Ivanovo region. First, we'll download all the necessary libraries for the work:

* `selenium` ‒ to automate your work in the browser
* `time` ‒ to add a delay
* `pandas` ‒ to save results to a dataframe

In [12]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager

from bs4 import BeautifulSoup
from pandas import DataFrame
from time import sleep


In [6]:
# constants
DEAL_TYPE: dict = {
    "deal_type": "rent"  # or "sale"
}
REGION: dict = {
    "region": 1,  # Moscow
}
FLAT_ROOMS: dict = {
    "room1": 1,
    "room2": 1,
    "room3": 1,
    "room4": 1,
    "room9": 1,  # studio
}
RENT_TYPE:dict = {
    "type": 4  # 4 - long period rent, 3 - for several months, 2 - short period rent, exclude the var - for buying
}
CIAN_NAKED = "https://www.cian.ru/cat.php?"


deal_type: str  = '&'.join(f"{k}={v}" for k, v in DEAL_TYPE.items())
region: str     = '&'.join(f"{k}={v}" for k, v in REGION.items())
flat_rooms: str = '&'.join(f"{k}={v}" for k, v in FLAT_ROOMS.items())
rent_type: str  = '&'.join(f"{k}={v}" for k, v in RENT_TYPE.items())



cian_link = (
    CIAN_NAKED + "&"
    + deal_type + "&"
    + "engine_version=2&"
    + region + "&"
    + flat_rooms + "&"
    + rent_type
)


# deal_type=rent&engine_version=2&offer_type=flat&region=1&room1=1&room2=1&room3=1&room9=1&type=4

https://www.cian.ru/cat.php?&deal_type=rent&engine_version=2&region=1&room1=1&room2=1&room3=1&room4=1&room9=1&type=4


In [16]:
options = webdriver.ChromeOptions()
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36")

driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
driver.maximize_window()
driver.get(cian_link)
# WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, "/html/body/div[1]/div/div/div[4]/div[1]/div/article/div[1]/div/div[1]/div/a")))
sleep(5)

# for i in (1, 2):
fld = driver.find_element('xpath', "/html/body/div[1]/div/div/div[4]/div[1]/div/article/div[1]/div/div[1]/div/a")
href = fld.get_attribute("href")
print(f"Found link: {href}")

driver.get(href)
print("Go with link")

sleep(5)


page_source = driver.page_source


soup = BeautifulSoup(page_source, "html.parser")

titles = soup.find_all("h1")
for title in titles:
    print("Title:", title.text.strip())


links = soup.find_all("a", href=True)
for link in links[:10]:
    print("Link:", link["href"])

#fld.click()

# /html/body/div[1]/div/div/div[4]/div[2]/div/article/div[1]/div/div[1]/div/a
# /html/body/div[1]/div/div/div[4]/div[34]/div/article/div[1]/div/div[1]/div/a


Found link: https://www.cian.ru/rent/flat/310496616/
Go with link
Title: Сдается 3-комн. квартира, 113 м²
Title: Расположение
Link: #
Link: //www.cian.ru
Link: /compare/
Link: //www.cian.ru/dialogs/
Link: https://www.cian.ru/rent/flat/favorites/
Link: https://www.cian.ru/razmestit-obyavlenie/?channel=submit_for_zero_rub
Link: /authenticate/?returnUrl=https://www.cian.ru/rent/flat/310496616/
Link: https://www.cian.ru/snyat/
Link: https://www.cian.ru/kupit/
Link: https://www.cian.ru/novostrojki/


Now let's write the function `get_uik_address()`, which takes two arguments as input, plot number and region, and returns a string with the address. To do this, copy the code from the previous session to the function body:

In [26]:
gtext = soup.get_text(separator="\n", strip=True)

list1 = gtext.split("\n")

soup.find_all('p')  # parameters



[<p class="a10a3f92e9--color_gray60_100--r_axa a10a3f92e9--lineHeight_6u--cedXD a10a3f92e9--fontWeight_normal--JEG_c a10a3f92e9--fontSize_16px--QNYmt a10a3f92e9--display_block--KYb25 a10a3f92e9--text--e4SBY a10a3f92e9--text_letterSpacing__0--cQxU5">Общая площадь</p>,
 <p class="a10a3f92e9--color_text-primary-default--vSRPB a10a3f92e9--lineHeight_6u--cedXD a10a3f92e9--fontWeight_normal--JEG_c a10a3f92e9--fontSize_16px--QNYmt a10a3f92e9--display_block--KYb25 a10a3f92e9--text--e4SBY a10a3f92e9--text_letterSpacing__0--cQxU5">113</p>,
 <p class="a10a3f92e9--color_gray60_100--r_axa a10a3f92e9--lineHeight_6u--cedXD a10a3f92e9--fontWeight_normal--JEG_c a10a3f92e9--fontSize_16px--QNYmt a10a3f92e9--display_block--KYb25 a10a3f92e9--text--e4SBY a10a3f92e9--text_letterSpacing__0--cQxU5">Площадь кухни</p>,
 <p class="a10a3f92e9--color_text-primary-default--vSRPB a10a3f92e9--lineHeight_6u--cedXD a10a3f92e9--fontWeight_normal--JEG_c a10a3f92e9--fontSize_16px--QNYmt a10a3f92e9--display_block--KYb25 a10

In [6]:
for i in range(10):
    print(random.randint(1, 10))

9
2
9
5
4
5
9
6
3
10


In [7]:
for i in range(10):
    print(random.uniform(1., 2.))

1.0521547378788774
1.783776643084079
1.2666689342728898
1.6095279195095382
1.1531589439323509
1.8259087191633654
1.58670992790143
1.291070445413658
1.405555743093474
1.049467865132934


In [8]:
def get_uik_address(n_uik, reg):
    
    driver.get("http://cikrf.ru/digital-services/naydi-svoy-izbiratelnyy-uchastok/")

#     fld = driver.find_element_by_xpath("/html/body/div[2]/div/div/button")
#     fld.click()

#     sleep(1.5)

    fld = driver.find_element('xpath', "/html/body/div[1]/div/div[2]/div/div[4]/form/div[1]/div[2]")
    fld.click()

    sleep(random.uniform(1., 2.))

    fld = driver.find_element('xpath', '/html/body/div[1]/div/div[2]/div/div[4]/form/div[3]/div[1]/div/span/span[1]/span/span[2]')
    fld.click()

    sleep(random.uniform(1., 2.))

    reg_field = driver.find_element('xpath', '/html/body/span/span/span[1]/input')
    reg_field.send_keys(reg)

    sleep(random.uniform(1., 2.))

    reg_field.send_keys(Keys.RETURN)

    sleep(random.uniform(1., 2.))

    uik_field = driver.find_element('xpath', '/html/body/div[1]/div/div[2]/div/div[4]/form/div[3]/div[2]/div/input')
    uik_field.click()
    uik_field.clear()
    uik_field.send_keys(n_uik)

    sleep(random.uniform(1., 2.))

    uik_field.send_keys(Keys.RETURN)
    
    sleep(random.uniform(1., 2.))
    
    my_page = BeautifulSoup(driver.page_source)
    p = filter(lambda x: x not in ('',None),[i.text for i in my_page.find_all('span',{'class':'address'})])
    p = next(p)
    
    if p is None or p=='':
        p = 'Адреса не нашлось'
        
    return p

Now let's try to take a few numbers of sections and see what happens in the loop. But let's play it safe by writing an expression with an exception, so that if the page doesn't contain an address or loads incorrectly, our code doesn't break and the loop doesn't break. If everything is good (there is an address), Python will save it (the" branch " with `try`), if everything is bad (there is no address in any form), Python will write an empty string instead (the branch with `except`) and move on. 

In [9]:
uiks = range(240, 245)

In [10]:
get_uik_address(244, "Ивановская область")

'155800, Ивановская область, город Кинешма, улица Григория Королева, дом 10, здание "Кинешемский политехнический колледж"'

In [11]:
addresses = []

for u in uiks:
    try:
        address = get_uik_address(u, "Ивановская область")
        print(address)
    except:
        address = ""
    addresses.append(address)
    print(u, address)

240 
155330, Ивановская область, городской округ Вичуга, город Вичуга, улица Желябова, дом 6, здание МБОУ ООШ №6
241 155330, Ивановская область, городской округ Вичуга, город Вичуга, улица Желябова, дом 6, здание МБОУ ООШ №6
155330, Ивановская область, городской округ Вичуга, город Вичуга, улица Ленинская, дом 26, здание МБУК "Клуб имени Шагова"
242 155330, Ивановская область, городской округ Вичуга, город Вичуга, улица Ленинская, дом 26, здание МБУК "Клуб имени Шагова"
155330, Ивановская область, городской округ Вичуга, город Вичуга, улица Ленинская, дом 26, здание МБУК "Клуб имени Шагова"
243 155330, Ивановская область, городской округ Вичуга, город Вичуга, улица Ленинская, дом 26, здание МБУК "Клуб имени Шагова"
155800, Ивановская область, город Кинешма, улица Григория Королева, дом 10, здание "Кинешемский политехнический колледж"
244 155800, Ивановская область, город Кинешма, улица Григория Королева, дом 10, здание "Кинешемский политехнический колледж"


It works! Let's create a list with all the numbers of polling stations in the Ivanovo region:

In [12]:
addresses

['',
 '155330, Ивановская область, городской округ Вичуга, город Вичуга, улица Желябова, дом 6, здание МБОУ ООШ №6',
 '155330, Ивановская область, городской округ Вичуга, город Вичуга, улица Ленинская, дом 26, здание МБУК "Клуб имени Шагова"',
 '155330, Ивановская область, городской округ Вичуга, город Вичуга, улица Ленинская, дом 26, здание МБУК "Клуб имени Шагова"',
 '155800, Ивановская область, город Кинешма, улица Григория Королева, дом 10, здание "Кинешемский политехнический колледж"']

**Important:** periodically open a browser window where Python searches for polling stations! This is not only pleasant (to see how everything is filled in in the search fields without our participation), but also useful: so you can notice if something went wrong. 

Let's create a dataframe from a dictionary whose keys are the names of table columns, and whose values are lists of elements in these columns.

In [13]:
df = pd.DataFrame({'uik': uiks, 'address': addresses})
#keys – names of future columns
#values - a list of values inside the future table

In [14]:
df.head()

,uik,address
0,240,
1,241,"155330, Ивановская область, городской округ Ви..."
2,242,"155330, Ивановская область, городской округ Ви..."
3,243,"155330, Ивановская область, городской округ Ви..."
4,244,"155800, Ивановская область, город Кинешма, ули..."


In [15]:
list(df.uik)

[240, 241, 242, 243, 244]

In [16]:
list(df.address)

['',
 '155330, Ивановская область, городской округ Вичуга, город Вичуга, улица Желябова, дом 6, здание МБОУ ООШ №6',
 '155330, Ивановская область, городской округ Вичуга, город Вичуга, улица Ленинская, дом 26, здание МБУК "Клуб имени Шагова"',
 '155330, Ивановская область, городской округ Вичуга, город Вичуга, улица Ленинская, дом 26, здание МБУК "Клуб имени Шагова"',
 '155800, Ивановская область, город Кинешма, улица Григория Королева, дом 10, здание "Кинешемский политехнический колледж"']

Save the table to a csv file:

In [17]:
df.to_csv('Ivanovo.csv', index=False, encoding='Windows-1251')

In [18]:
df.to_excel('Ivanovo.xlsx', index=False)

<ipython-input-18-c551dbfda63d>:1: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.2.9' currently installed).
  df.to_excel('Ivanovo.xlsx', index=False)


In [19]:
driver.close()